<a href="https://colab.research.google.com/github/stevenpollack/camelcup/blob/master/camelup_gcp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import itertools as it
import collections, copy

class Camel:
  def __init__(self, color, location=-1, already_moved=False):
    self.color = color
    self.location = location
    self.already_moved = already_moved
    return;

  def __repr__(self):
    return self.__str__()    
  def __str__(self):
    return "{} camel is on space #{}".format(self.color, self.location)
  

class Space:
  ''' The space class is meant to model a given space on the playing board.
  A space can either have:
  * a oasis tile (effect=+1) or a mirage tile (effect=-1),
  XOR 
  * a list of a camels -- ordering is important here: [A, B, C] indicates that
  A sits atop B which sits atop C

  * `location` is meant to index the space on the board. location == 0 is reserved
  for the "podium" -- the space where camels are placed when the cross the finish
  line.

  For performance reasons, type and value checks are omitted here. This class isn't
  meant to be accessible by end-users, so those checks are meant to be performed
  during parameter ingestions, later.
  '''
  def __init__(self, location, effect=0):
    self.camels = []
    self.effect = effect
    self.location = location
    return;

  def __str__(self):
    return self.__repr__()
  def __repr__(self):
    return "<Space: location {}, {}, and effect {}>".format(self.location, self.camels, self.effect)

  def add_tile(self, effect):
    self.effect = effect
    return;

  ''' The add_camels_to_* methods are where camel objects get their location
  parameters updated, and where spaces get their camels arrays updated.

  * camels must be a list-like object of camel objects. 

  We need to preserve the orderings of camel stacks, so when we add one stack
  to the top of another, we want to extend the new stack (camels) by the
  already-positioned stack (self.camels). When we add one stack to the bottom of
  another (from mirage tiles), we can append the new stack to the back of the
  already-positioned stack.
  '''
  def add_camels_to_top(self, camels):
    for camel in camels:
      camel.location = self.location
    
    # use .extend() for performance
    if self.camels:
      camels.extend(self.camels)
    
    self.camels = camels
    return;

  def add_camels_to_bottom(self, camels):
    for camel in camels:
      self.camels.append(camel)
      camel.location = self.location
    return;


class Board:
  def __init__(self, state_of_play):
    ''' Board is initialized via a dict/JSON object, `state_of_play`, which must
    have key `camels`, and can have option key `tiles`.
    * camels: an array of dictionaries with keys {color[str], location[int], already_moved[bool]}.
      @ location must be an int between 1 and num_of_spaces (16)
      @ no two camels should have the same color
      @ camels that share the same location will be interpreted as being stacked
        from left-to-right. Meaning, the first camel signaled at a given location
        will be taken as the lead camel.
    * tiles: an array of dictionaries with keys {location[int], effect[int]}.
      @ location must be an int between 1 and num_of_spaces (16)
      @ no two tiles should have the same location
      @ effect must be in [-1,0,1]
    
    Board has a few internal properties:
    * moved_camels: an array holding the camel objects that have been declared as
    "already moved" in the state of play
    * ready_camels: an array holding the camel objects that have yet to move
    * num_of_spaces: the (int) number of spaces on the board (default == 16)
    * spaces: an array holding Space objects that each represent a particular space,
    including the "podium" which is spaces[0], where a winning camel stack is taken
    once it crosses the last space.
    '''
    # there are 16 spaces on the board, and we'll reserve the 0-index space for 
    # where camels go when the race is over
    self.num_of_spaces = 16 # standard board size
    self.moved_camels, self.ready_camels = [], []

    # initialize the board with empty spaces
    self.spaces = []
    for i in range(self.num_of_spaces+1): # we want the 0th space to be reserved for the winners
      self.spaces.append(Space(location=i))

    self.parse_state_of_play(state_of_play)

    return;

  def __repr__(self):
    [print(space) for space in self.spaces]
    return "."

  def get_camel(self, color):
    # only works for camels in ready_camels
    for camel in self.ready_camels:
      if camel.color == color:
        return camel

  def parse_state_of_play(self, state_of_play):
    ''' state_of_play is a dictionary with keys in [camels, tiles].
    see __init__ for assumptions about state_of_play
    '''
    for camel_dict in state_of_play['camels']:
      camel = Camel(camel_dict['color'], camel_dict['location'])
      # add camel to the space, building the stack from top-down
      self.spaces[camel.location].add_camels_to_bottom([camel]) # this method needs an array of camels to be passed in
      # sort camels depending on whether they've moved or not
      if camel_dict['already_moved']:
        self.moved_camels.append(camel)
      else:
        self.ready_camels.append(camel)
    
    for tile in state_of_play['tiles']:
      if tile['effect'] != 0:
        self.spaces[tile['location']].effect = tile['effect']
    return;

  def clear(self):
    self.moved_camels.clear()
    self.ready_camels.clear()
    for space in self.spaces:
      space.camels.clear()
      space.effect = 0


  def resolve_roll(self, camel, roll, verbose=False):

    future_location = camel.location + roll

    # we need to figure out if this camel has camels on its back, and then
    # remove them from this space (and then move them to their future space)
    current_space = self.spaces[camel.location]
    camel_index = current_space.camels.index(camel)
    camel_stack = [current_space.camels.pop(0) for x in range(camel_index+1)]

    if verbose:
      print(camel_stack)

    if future_location > self.num_of_spaces:  # game is over

      final_space = self.spaces[0] #this is the "podium"
      final_space.add_camels_to_top(camel_stack)
      
      # need to end the round
      if verbose:
        print("game ending!")

    else: # no camels have crossed the finish line:
      # check for any tiles that the camel stack may land on, and then bounce
      # off of, and determine the final space, after all tile effects.

      # simulate bouncing forwards or backwards across potentially many mirages or oases
      final_space = self.spaces[future_location]
      tile_effect = 0 # this will get updated in the loop, if necessary. It's == final_space.effect

      while final_space.effect != 0:
        # check for tile effects
        tile_effect = final_space.effect
        future_location += tile_effect # update
        final_space = self.spaces[future_location]
    
      if tile_effect == -1: # append camel to bottom of stack
        final_space.add_camels_to_bottom(camel_stack)
      else: # tile_effect >= 0 -- # pop camel atop stack
        final_space.add_camels_to_top(camel_stack)

      if verbose:
        print("ready camels: {}".format(self.ready_camels))
        #[print(camel) for camel in camel_stack]

    return final_space

  def get_leg_ranking(self):
    ''' this returns an array of strings indicating the color of the camel, in
    relative ranking. 
    * self.leg_ranking[0] == first place, 
    * self.leg_ranking[1] == second place, etc.
    '''

    self.leg_ranking = []
    # we want to iterate around the board, backwards, aggregating the camel stacks
    # and then extracting the camel colors
    reversed_indices = [0]
    reversed_indices.extend(range(self.num_of_spaces, 0, -1))

    # it.chain can be used to pythonically do this across *camel_stacks, but
    # it's slightly less performant
    for index in reversed_indices:
      camel_stack = self.spaces[index].camels
      if camel_stack:
        for camel in camel_stack:
          self.leg_ranking.append(camel.color)

    return self.leg_ranking    

################################################################################

def normalize_counts(counter,sig_figs=2, factor=0):
  if factor == 0:
    factor = sum(counter.values())
  output = copy.deepcopy(counter)
  for k,v in counter.items():
    output[k] = round(v/factor, sig_figs)
  return output

def calculate_all_outcomes(state_of_play, normalize=True, sig_figs=2, verbose=False, **kwargs):

  board = Board(state_of_play)
  if verbose:
    print(board)

  # generate all possible events from ready camels
  n_ready_camels, n_moved_camels = len(board.ready_camels), len(board.moved_camels)
  
  outcomes = [[] for i in range(n_ready_camels + n_moved_camels)] # this is should be a C x N matrix

  possible_camel_orderings = it.permutations([camel.color for camel in board.ready_camels])
  possible_dice_throws = it.product([1,2,3], repeat=n_ready_camels)
  possible_events = it.product(possible_camel_orderings, possible_dice_throws) # len(possible_events) == N

  # the permutations of camels and rolls are "unzipped", so we'll want to
  # zip together camels and their corresponding rolls. Then, we'll need to resolve
  # each (camel, roll) "event",
  for camels, rolls in possible_events:
    for camel_color, roll in zip(camels, rolls):
      camel = board.get_camel(camel_color)
      resulting_space = board.resolve_roll(camel, roll, verbose=verbose)
      
      if resulting_space.location == 0: # game is over
        break

    leg_ranking = board.get_leg_ranking()
    for idx, camel_color in enumerate(leg_ranking):
      outcomes[idx].append(camel_color)

    # reset state of board for next iteration
    board.clear()
    board.parse_state_of_play(state_of_play)
  
  if normalize:
    freqs = []
    num_outcomes = len(outcomes[0])
    for outcome in outcomes:
      counter = collections.Counter(outcome)
      freqs.append(normalize_counts(counter, sig_figs, num_outcomes))
    return freqs

  return outcomes

def process_request(request):
  """Responds to any HTTP request.
  Args:
      request (flask.Request): HTTP request object.
      The request must have a "state_of_play" key with value being
      a state_of_play dictionary as defined in the Board class.
      Optionally, one may pass (sig_figs, [int]) which indicates the number
      of digits left on the probabilities, after rounding.
  Returns:
      A JSON string with keys "computational_duration" (the time it took to
      compute all probabilities) and "probs" an array of dictionaries,
      ordered such that the first dictionary yields probabilities of coming
      in 1st place, the second, 2nd place, etc.
  """
  request_json = request.get_json()
  
  state_of_play = request_json.get('state_of_play')
  sig_figs = request_json.get('sig_figs')
  
  
  print(request_json)
  
  t0 = time.time()
  freqs = calculate_all_outcomes(**request_json)
  t1 = time.time()
  
  return json.dumps({"computational_duration": t1-t0, "probs": freqs})

In [0]:
# test object for cloud platforms
{
"state_of_play": {
    "camels": [
          {"color": "orange", "location": 3, "already_moved": false},
          {"color": "blue", "location": 1, "already_moved": false},
          {"color": "yellow", "location": 1, "already_moved": false},
          {"color": "green", "location": 1, "already_moved": false},
          {"color": "white", "location": 1, "already_moved": false}
           ],
    "tiles": [
        {"location": 16, "effect": 0},
        {"location": 3, "effect": 0}
         ]
    
},
"sig_figs": 5,
"non-sense": false
}